In [1]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import matplotlib.cm as cm

# 
import numpy as np
import os
from tqdm import trange
import parmap
import glob
from sklearn.decomposition import PCA
#import umap
import seaborn as sns
import pandas as pd

import pickle
# 
from tqdm import tqdm


import sklearn.experimental
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# 
from scipy.io import loadmat
import scipy
import scipy.ndimage
# 

import numba
from numba import jit
import parmap


Autosaving every 180 seconds


In [2]:

#######################################
#######################################
#######################################
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.backend_bases import MouseButton
from matplotlib.path import Path


class ManualCluster(object):
    def __init__(self, fname):
        
        self.fig = plt.figure()
        self.ax = plt.subplot()

        self.x0 = None
        self.y0 = None
        self.x1 = None
        self.y1 = None
        self.cluster_locs = []
        self.cluster_ids = []
        self.coords = []
        self.fname = fname
        
        X_fit = np.load(fname)
        self.X_fit = X_fit.copy()
        
        #
        self.data = X_fit
        
        # keep track of unit ids
        self.data_ids = np.arange(X_fit.shape[0])
        
        #
        self.draw()
        
        #self.ax.add_patch(self.rect)
        self.ax.figure.canvas.mpl_connect('button_press_event', self.on_press)
        self.ax.figure.canvas.mpl_connect('button_release_event', self.on_release)

    def points_inside_polygon(self):
    
        #FIND points inside polygon
        clustered_ids = []
        clustered_ids_global = []
        clustered_locs = []
        bbPath = Path(np.array(self.coords))
        for k,d in enumerate(self.data):
            if bbPath.contains_point(d):
                clustered_ids_global.append(self.data_ids[k])
                clustered_ids.append(k)
                clustered_locs.append(self.data[k])

        # Delete the clustered_pts indexes from 
        self.data = np.delete(self.data, clustered_ids, axis=0)
        self.data_ids = np.delete(self.data_ids, clustered_ids, axis=0)
        
        # 
        self.clustered_ids = np.array(clustered_ids)
        
        #
        self.clustered_locs = np.array(clustered_locs)
        
        #
        self.clustered_ids_global = np.array(clustered_ids_global)

        #

    def draw(self):
         self.ax.scatter(self.data[:,0], self.data[:,1],
                         s=1,
                         alpha=0.5,
                         c='blue')
        
        
    def on_press(self, event):
        #print ('press')
        
        if event.button is MouseButton.LEFT:
            #print ("LEFT")

            # 
            self.x0 = event.xdata
            self.y0 = event.ydata
            #print ("adding pts: ", self.x0, self.y0)
            self.coords.append([self.x0, self.y0])
            
            # 
            self.ax.scatter([self.x0, self.x0],
                            [self.y0, self.y0],
                            c='black')

        if event.button is MouseButton.RIGHT:
            #print ("RIGHT")
            for k in range(len(self.coords)-1):
                self.ax.plot([self.coords[k][0], self.coords[k+1][0]],
                             [self.coords[k][1], self.coords[k+1][1]],
                        c='black')
            # 
            self.ax.plot([self.coords[0][0], self.coords[-1][0]],
                         [self.coords[0][1], self.coords[-1][1]],
                    c='black')

            # 
            self.points_inside_polygon()
            
            #
            self.cluster_ids.append(self.clustered_ids_global)
            print ("# of clustered points: ", self.clustered_ids_global.shape)
            
            # 
            self.cluster_locs.append(self.clustered_locs)
            print ("# of clusters: ", len(self.clustered_locs))
            
            # 
            print ("all_data remaining: ", self.data.shape)
  
            # rset everything
            self.ax.cla()
            self.draw()
            self.coords=[]
            
            
        if event.button is MouseButton.MIDDLE:
            
            print ("SAVING clustering data")
            np.savez(self.fname.replace('npy','npz'),
                     data = self.X_fit,
                     cluster_locs = self.cluster_locs,
                     cluster_ids = self.cluster_ids)
            
    # 
    def on_release(self, event):
        #print ('release')
        #self.x1 = event.xdata
        #self.y1 = event.ydata
        #self.rect.set_width(self.x1 - self.x0)
        #self.rect.set_height(self.y1 - self.y0)
        #self.rect.set_xy((self.x0, self.y0))
        self.ax.figure.canvas.draw()

#X_fit = np.load('/home/cat/umap_minDist_0.0_nNeighbor_200.npy')
#X_fit = np.load('/home/cat/umap_x_fit_min_duration_50_dataType_discretized.npy')
#X_fit = np.load('/home/cat/umap_x_fit_min_duration_50_dataType_histograms.npy')



fname = '/home/cat/Xfit_unique.npy'
a = ManualCluster(fname)


In [130]:
d = np.load('/home/cat/res.npz')
X_fit = d['X_fit']
np.save('/home/cat/Xfit_unique.npy', X_fit)
all_hist = d['X']


In [134]:
######################################################
########### VISUALIZE CLUSTER RESULTS ################
######################################################
# 
d = np.load(fname.replace('npy','npz'), allow_pickle=True)

data = d['data']
ids = d['cluster_ids']
print (data.shape)
fig=plt.figure()
for k in range(ids.shape[0]):
    dd = data[ids[k]]
    plt.scatter(dd[:,0], dd[:,1],
               s=1,
               alpha=.5)
    x = np.mean(dd[:,0])
    y = np.mean(dd[:,1])
    plt.text(x,y,str(k),fontsize=20)
    
    
plt.show()

(296, 2)


In [138]:
############################################
######### VISUALIZE HISTOGRAM DATA #########
############################################

# 
if False:
    d = np.load('/media/cat/1TB/dan/cohort1/slp/all_continuous_0_min_duration8.npz')
    all_hist = d['all_hist']
print (all_hist.shape)

# load clustered ids
fname = '/home/cat/Xfit_unique.npy'
d = np.load(fname.replace('.npy','.npz'), allow_pickle=True)

# 
data = d['data']
cluster_ids = d['cluster_ids']
print (cluster_ids.shape)

#
s = 0
#cls = [0,1,2,5,6,7,12,3,4,8,9,10,11,13,14,15,16,17]
uniq_vectors = []
for k in range(cluster_ids.shape[0]):
#for k in cls:
    idx = cluster_ids[k]
    temp = all_hist[idx]
    uniq = np.unique(temp,axis=0)
    med = np.median(temp,axis=0)
    #med = np.mean(temp,axis=0)

    print (k, temp.shape[0], " uniq: ", uniq.shape[0], "  med angle: ", np.round(med[:3],3),
          "  med acc: ", np.round(med[3:],3))
    s+=temp.shape[0]
    
    uniq_vectors.append(uniq)
print ("total:", s)

uniq_vectors = np.array(uniq_vectors).squeeze()
print (uniq_vectors)

    

(296, 5)
(7,)
0 45  uniq:  45   med angle:  [0.25 0.25 0.25]   med acc:  [0. 1.]
1 43  uniq:  43   med angle:  [0.25 0.25 0.25]   med acc:  [0.5 0.5]
2 45  uniq:  45   med angle:  [0.25 0.25 0.25]   med acc:  [0.167 0.833]
3 45  uniq:  45   med angle:  [0.25 0.25 0.25]   med acc:  [0.333 0.667]
4 38  uniq:  38   med angle:  [0.25  0.375 0.375]   med acc:  [0.833 0.167]
5 42  uniq:  42   med angle:  [0.312 0.312 0.25 ]   med acc:  [0.667 0.333]
6 38  uniq:  38   med angle:  [0.312 0.375 0.25 ]   med acc:  [1. 0.]
total: 296
[array([[0.   , 0.   , 1.   , 0.   , 1.   ],
       [0.   , 0.125, 0.875, 0.   , 1.   ],
       [0.   , 0.25 , 0.75 , 0.   , 1.   ],
       [0.   , 0.375, 0.625, 0.   , 1.   ],
       [0.   , 0.5  , 0.5  , 0.   , 1.   ],
       [0.   , 0.625, 0.375, 0.   , 1.   ],
       [0.   , 0.75 , 0.25 , 0.   , 1.   ],
       [0.   , 0.875, 0.125, 0.   , 1.   ],
       [0.   , 1.   , 0.   , 0.   , 1.   ],
       [0.125, 0.   , 0.875, 0.   , 1.   ],
       [0.125, 0.125, 0.75 , 0

In [91]:
#
s = 0
cls = [0,1,2,5,6,7,12,3,4,8,9,10,11,13,14,15,16,17]
uniq_vectors = []
for k in range(cluster_ids.shape[0]):
#for k in cls:
    idx = cluster_ids[k]
    temp = all_hist[idx]
    uniq, counts = np.unique(temp, axis=0, return_counts=True)
    
    sort = np.argsort(counts)[::-1]
    counts = counts[sort]
    uniq = uniq[sort]
    med = np.mean(temp,axis=0)
    #med = np.mean(temp,axis=0)

    print (k, temp.shape[0], " uniq: ", uniq.shape[0], "  med angle: ", np.round(med[:3],3),
          "  med acc: ", np.round(med[3:],3))
    print ("counts: ", counts[:5], "  data; ", uniq[:5])
    s+=temp.shape[0]
    
    uniq_vectors.append(uniq)
    print ('')
print ("total:", s)

uniq_vectors = np.array(uniq_vectors).squeeze()
print (uniq_vectors)





0 102934  uniq:  1   med angle:  [0. 1. 0.]   med acc:  [1. 0.]
counts:  [102934]   data;  [[0. 1. 0. 1. 0.]]

1 12081  uniq:  1   med angle:  [0. 1. 0.]   med acc:  [0.167 0.833]
counts:  [12081]   data;  [[0.         1.         0.         0.16666667 0.8333333 ]]

2 12694  uniq:  1   med angle:  [0. 1. 0.]   med acc:  [0.5 0.5]
counts:  [12694]   data;  [[0.  1.  0.  0.5 0.5]]

3 12291  uniq:  1   med angle:  [0. 1. 0.]   med acc:  [0.667 0.333]
counts:  [12291]   data;  [[0.         1.         0.         0.6666667  0.33333334]]

4 13503  uniq:  1   med angle:  [0. 1. 0.]   med acc:  [0.333 0.667]
counts:  [13503]   data;  [[0.         1.         0.         0.33333334 0.6666667 ]]

5 7455  uniq:  1   med angle:  [0.125 0.875 0.   ]   med acc:  [1. 0.]
counts:  [7455]   data;  [[0.125 0.875 0.    1.    0.   ]]

6 8142  uniq:  1   med angle:  [0.    0.875 0.125]   med acc:  [1. 0.]
counts:  [8142]   data;  [[0.    0.875 0.125 1.    0.   ]]

7 7438  uniq:  1   med angle:  [0. 1. 0.]   me

In [83]:
def get_index(array, feature_array):

    idx = np.sum(np.equal(feature_array, array), axis=1)
    idx = np.where(idx==len(array))[0]

    return idx

colors = plt.cm.gist_ncar(np.linspace(0,1,20))
print (uniq_vectors.shape)
ax=plt.subplot()
offset = 10000
for k in range(60*25):
    
    temp = all_hist[k+offset]
    print ("temp: ", temp)
    #print (uniq_vectors)
    idx = get_index(temp, uniq_vectors)
    print ("idx", idx)
    t = k/25
    if idx.shape[0]>0:
        ax.axvspan(t,t+0.99, alpha=1, color=colors[idx[0]])
        #plt.scatter(k, 0, c=colors[idx[0]])
    else:
        ax.axvspan(t,t+0.99, alpha=1, color='black')
plt.xlabel("Time (sec)")
plt.ylabel("Behaviors")
plt.yticks([])
plt.xlim(0,t)
plt.ylim(0,1)

plt.show()

(18, 5)
temp:  [0.         0.875      0.125      0.6666667  0.33333334]
idx [12]
temp:  [0.125      0.75       0.125      0.8333333  0.16666667]
idx []
temp:  [0.125 0.875 0.    1.    0.   ]
idx [7]
temp:  [0. 1. 0. 1. 0.]
idx [0]
temp:  [0.    0.875 0.125 1.    0.   ]
idx [8]
temp:  [0.    0.875 0.125 1.    0.   ]
idx [8]
temp:  [0.   0.75 0.25 1.   0.  ]
idx []
temp:  [0.  0.5 0.5 1.  0. ]
idx []
temp:  [0.    0.375 0.625 1.    0.   ]
idx []
temp:  [0.    0.375 0.625 1.    0.   ]
idx []
temp:  [0.    0.625 0.375 1.    0.   ]
idx []
temp:  [0.  0.5 0.5 1.  0. ]
idx []
temp:  [0.25  0.625 0.125 1.    0.   ]
idx []
temp:  [0.375 0.375 0.25  1.    0.   ]
idx []
temp:  [0.25 0.75 0.   1.   0.  ]
idx []
temp:  [0.25 0.75 0.   1.   0.  ]
idx []
temp:  [0.5 0.5 0.  1.  0. ]
idx []
temp:  [0.5 0.5 0.  1.  0. ]
idx []
temp:  [0.5   0.375 0.125 1.    0.   ]
idx []
temp:  [0.125 0.875 0.    1.    0.   ]
idx [7]
temp:  [0.25 0.75 0.   1.   0.  ]
idx []
temp:  [0.125 0.875 0.    1.    0.   ]
idx [

temp:  [0. 1. 0. 1. 0.]
idx [0]
temp:  [0. 1. 0. 1. 0.]
idx [0]
temp:  [0. 1. 0. 1. 0.]
idx [0]
temp:  [0. 1. 0. 1. 0.]
idx [0]
temp:  [0. 1. 0. 1. 0.]
idx [0]
temp:  [0. 1. 0. 1. 0.]
idx [0]
temp:  [0. 1. 0. 1. 0.]
idx [0]
temp:  [0. 1. 0. 1. 0.]
idx [0]
temp:  [0. 1. 0. 1. 0.]
idx [0]
temp:  [0.125 0.875 0.    1.    0.   ]
idx [7]
temp:  [0. 1. 0. 1. 0.]
idx [0]
temp:  [0. 1. 0. 1. 0.]
idx [0]
temp:  [0. 1. 0. 1. 0.]
idx [0]
temp:  [0. 1. 0. 1. 0.]
idx [0]
temp:  [0. 1. 0. 1. 0.]
idx [0]
temp:  [0. 1. 0. 1. 0.]
idx [0]
temp:  [0. 1. 0. 1. 0.]
idx [0]
temp:  [0. 1. 0. 1. 0.]
idx [0]
temp:  [0. 1. 0. 1. 0.]
idx [0]
temp:  [0. 1. 0. 1. 0.]
idx [0]
temp:  [0. 1. 0. 1. 0.]
idx [0]
temp:  [0. 1. 0. 1. 0.]
idx [0]
temp:  [0. 1. 0. 1. 0.]
idx [0]
temp:  [0. 1. 0. 1. 0.]
idx [0]
temp:  [0.    0.875 0.125 1.    0.   ]
idx [8]
temp:  [0.    0.875 0.125 1.    0.   ]
idx [8]
temp:  [0.         0.875      0.125      0.8333333  0.16666667]
idx []
temp:  [0.         0.875      0.125      0.6666667  

temp:  [0.375      0.25       0.375      0.33333334 0.6666667 ]
idx []
temp:  [0.         0.5        0.5        0.16666667 0.8333333 ]
idx []
temp:  [0.25       0.375      0.375      0.16666667 0.8333333 ]
idx []
temp:  [0.125      0.25       0.625      0.16666667 0.8333333 ]
idx []
temp:  [0.         0.         1.         0.16666667 0.8333333 ]
idx []
temp:  [0.  0.5 0.5 0.  1. ]
idx []
temp:  [0.         0.25       0.75       0.16666667 0.8333333 ]
idx []
temp:  [0.75 0.25 0.   0.5  0.5 ]
idx []
temp:  [0.75 0.25 0.   0.5  0.5 ]
idx []
temp:  [0.375 0.5   0.125 0.5   0.5  ]
idx []
temp:  [0.125      0.625      0.25       0.33333334 0.6666667 ]
idx []
temp:  [0.125      0.5        0.375      0.16666667 0.8333333 ]
idx []
temp:  [0.25 0.5  0.25 0.   1.  ]
idx []
temp:  [0.375 0.375 0.25  0.    1.   ]
idx []
temp:  [0.125      0.625      0.25       0.16666667 0.8333333 ]
idx []
temp:  [0.125      0.5        0.375      0.16666667 0.8333333 ]
idx []
temp:  [0.         0.5        0.5      

temp:  [0.375      0.625      0.         0.8333333  0.16666667]
idx []
temp:  [0.5 0.5 0.  1.  0. ]
idx []
temp:  [0.375 0.625 0.    1.    0.   ]
idx []
temp:  [0.25       0.625      0.125      0.8333333  0.16666667]
idx []
temp:  [0.5        0.5        0.         0.6666667  0.33333334]
idx []
temp:  [0.375 0.625 0.    0.5   0.5  ]
idx []
temp:  [0.25       0.75       0.         0.33333334 0.6666667 ]
idx []
temp:  [0.         1.         0.         0.33333334 0.6666667 ]
idx [5]
temp:  [0.         1.         0.         0.16666667 0.8333333 ]
idx [1]
temp:  [0.         1.         0.         0.8333333  0.16666667]
idx [4]
temp:  [0.    0.875 0.125 1.    0.   ]
idx [8]
temp:  [0.         1.         0.         0.16666667 0.8333333 ]
idx [1]
temp:  [0.         0.75       0.25       0.16666667 0.8333333 ]
idx []
temp:  [0.         0.875      0.125      0.33333334 0.6666667 ]
idx [13]
temp:  [0.         1.         0.         0.16666667 0.8333333 ]
idx [1]
temp:  [0.         1.         0.     

temp:  [0.   0.75 0.25 1.   0.  ]
idx []
temp:  [0.   0.75 0.25 1.   0.  ]
idx []
temp:  [0.   0.75 0.25 1.   0.  ]
idx []
temp:  [0.   0.75 0.25 1.   0.  ]
idx []
temp:  [0.   0.75 0.25 1.   0.  ]
idx []
temp:  [0.    0.875 0.125 1.    0.   ]
idx [8]
temp:  [0. 1. 0. 1. 0.]
idx [0]
temp:  [0. 1. 0. 1. 0.]
idx [0]
temp:  [0. 1. 0. 1. 0.]
idx [0]
temp:  [0.    0.875 0.125 1.    0.   ]
idx [8]
temp:  [0. 1. 0. 1. 0.]
idx [0]
temp:  [0. 1. 0. 1. 0.]
idx [0]
temp:  [0.125 0.75  0.125 1.    0.   ]
idx []
temp:  [0.125 0.875 0.    1.    0.   ]
idx [7]
temp:  [0.125 0.875 0.    1.    0.   ]
idx [7]
temp:  [0. 1. 0. 1. 0.]
idx [0]
temp:  [0.125 0.875 0.    1.    0.   ]
idx [7]
temp:  [0.    0.875 0.125 1.    0.   ]
idx [8]
temp:  [0. 1. 0. 1. 0.]
idx [0]
temp:  [0. 1. 0. 1. 0.]
idx [0]
temp:  [0. 1. 0. 1. 0.]
idx [0]
temp:  [0.    0.875 0.125 1.    0.   ]
idx [8]
temp:  [0. 1. 0. 1. 0.]
idx [0]
temp:  [0. 1. 0. 1. 0.]
idx [0]
temp:  [0. 1. 0. 1. 0.]
idx [0]
temp:  [0. 1. 0. 1. 0.]
idx [0]
temp

temp:  [0.125 0.875 0.    0.5   0.5  ]
idx [14]
temp:  [0.         1.         0.         0.6666667  0.33333334]
idx [2]
temp:  [0.         1.         0.         0.6666667  0.33333334]
idx [2]
temp:  [0.         1.         0.         0.6666667  0.33333334]
idx [2]
temp:  [0.         0.875      0.125      0.6666667  0.33333334]
idx [12]
temp:  [0.         0.875      0.125      0.8333333  0.16666667]
idx []
temp:  [0.    0.875 0.125 1.    0.   ]
idx [8]
temp:  [0.25       0.75       0.         0.8333333  0.16666667]
idx []
temp:  [0.25       0.75       0.         0.6666667  0.33333334]
idx []
temp:  [0.5   0.375 0.125 0.5   0.5  ]
idx []
temp:  [0.5        0.5        0.         0.33333334 0.6666667 ]
idx []
temp:  [0.625      0.375      0.         0.33333334 0.6666667 ]
idx []
temp:  [0.625      0.375      0.         0.16666667 0.8333333 ]
idx []
temp:  [0.375      0.375      0.25       0.16666667 0.8333333 ]
idx []
temp:  [0.125      0.5        0.375      0.16666667 0.8333333 ]
idx []
te

In [10]:
d = np.load('/media/cat/1TB/dan/cohort1/slp/all_continuous_0_min_duration25.npz')
all_hist = d['all_hist']

In [11]:
print (all_hist.shape)

(192964, 5)


In [22]:
import umap

#data = np.load('/home/cat/S_uniq_sym.npy')
#X_subsampled = data

import umap


# 
X_subsampled = all_hist.copy()#[::10]

if True:
    uniq = np.unique(X_subsampled, axis=0)
    
    X_subsampled = uniq
    

print ("fitting umap on" ,X_subsampled.shape)
if False:
    noise = np.random.rand(X_subsampled.shape[0], X_subsampled.shape[1])/4
    print ("noise mean: ", noise.mean())

    #for k in range(6):
    #    plt.subplot(2,3,k+1)
    #    
    #    
    #    
    print ("noise: ", noise.shape)
    X_subsampled += noise

#
# from sklearn import decomposition
# pca = decomposition.PCA(n_components=2)

# X_fit = pca.fit_transform(X_subsampled)
# print (X_fit.shape)

n_neighbors = 100
min_dist = 0.8
umap = umap.UMAP(n_components=2,
                 n_neighbors=n_neighbors,
                 min_dist = min_dist,
                 init='random',
                 random_state=0)

# 
X_fit = umap.fit_transform(X_subsampled)

# 
print (X_fit.shape)
plt.scatter(X_fit[:,0], X_fit[:,1],
            s=10,
            alpha=.2)
plt.title(str(n_neighbors)+ " "+str(min_dist))

# np.savez('/home/cat/res.npz',
#        X = X_subsampled,
#         X_fit = X_fit)
plt.show()

fitting umap on (3527, 5)
(3527, 2)


In [9]:
# print (all_hist[:100])

[[0.         1.         0.         1.         0.         0.        ]
 [0.         1.         0.         1.         0.         0.        ]
 [0.         1.         0.         1.         0.         0.        ]
 [0.         1.         0.         1.         0.         0.        ]
 [0.         1.         0.         1.         0.         0.        ]
 [0.         1.         0.         1.         0.         0.        ]
 [0.         1.         0.         1.         0.         0.        ]
 [0.         1.         0.         1.         0.         0.        ]
 [0.         1.         0.         1.         0.         0.        ]
 [0.         1.         0.         1.         0.         0.        ]
 [0.         1.         0.         1.         0.         0.        ]
 [0.         1.         0.         1.         0.         0.        ]
 [0.         1.         0.         1.         0.         0.        ]
 [0.         1.         0.         1.         0.         0.        ]
 [0.         1.         0.        

In [55]:
from klaus_emd_tools import plot_angle_acceleration_distributions
min_duration=8
animal_ids = [0]
#min_duration = 50
plot_angle_acceleration_distributions(animal_ids, min_duration)

angles:  (264580, 8)



In [113]:
#################################################
############ VISUALIZE RAW DATA #################
#################################################

# load continuous data
d = np.load('/home/cat/all_continuous_animalID_0.npz', allow_pickle=True)
angles = d['angles']
acc_ap = d['acc_ap']
acc_ml = d['acc_ml']
print (angles.shape, angles[0])

# 
d = np.load('/home/cat/all_discretized_25.npy')
print ("d: ", d.shape)
print (d[0])

# load clustered ids
d = np.load('/home/cat/cluster.npz', allow_pickle=True)
data = d['data']
cluster_ids = d['cluster_ids']
print (cluster_ids.shape)

#  
data_null = np.array([[0,0],[0,-1]])

# 
movs = []
for cluster_id in range(cluster_ids.shape[0]):


    colors = plt.cm.viridis(np.linspace(0,1,angles[frame_id].shape[0]))


    all_movement = []
    for id_ in cluster_ids[cluster_id]:

        loc_ap = 0
        loc_ml = 0
        frame_movement = []
        
        # compute angles + locations from accelerations
        for k in range(angles[id_].shape[0]):


            # Compute 
            loc_ap += acc_ap[id_][k]
            loc_ml += acc_ml[id_][k]

            frame_movement.append([loc_ap, loc_ml])
    #         plt.scatter(loc_ml, loc_ap,
    #                    color=colors[k])

        all_movement.append(frame_movement)
    movs.append(all_movement)
    
    
movs = np.array(movs)
print (movs.shape)

(306873, 13) [0.0028484  0.06521069 0.06793789 0.0636612  0.06760117 0.07107042
 0.07107042 0.07109863 0.07109863 0.07184006 0.07190161 0.0722482
 0.07169238]
d:  (272450, 46)
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [111]:
#########################################
alldata = np.load('/media/cat/1TB/dan/cohort1/slp/vecs_ego_animalID_0_duration_25.npy')
print (alldata.shape)

# load clustered ids
d = np.load(fname.replace('npy','npz'), allow_pickle=True)
data = d['data']
cluster_ids = d['cluster_ids']
print ("# clusters: ", cluster_ids.shape)

fig = plt.figure()
colors = plt.cm.viridis(np.linspace(0,1,25))
for cluster_id in range(cluster_ids.shape[0]):
    plt.subplot(3,5,cluster_id+1)
    # 
    xlims = np.zeros(2)
    ylims = np.zeros(2)
    ave = []
    for k in range(100):
        idx_abs = np.random.choice(cluster_ids[cluster_id])

        f= 0
        x = alldata[idx_abs,:,f,0]
        y = alldata[idx_abs,:,f,1]
        #x = x-x[0]
        #y = y-y[0]
        
        plt.scatter(x,y,
                    c=colors,
                   alpha=.1)
        
        
        for p in range(0,x.shape[0]-1,1):
            plt.plot([x[p],x[p+1]],
                     [y[p],y[p+1]],
                     c=colors[p])
            
        xlims[0] = max(xlims[0], np.max(x))
        xlims[1] = min(xlims[1], np.min(x))
        ylims[0] = max(ylims[0], np.max(y))
        ylims[1] = min(ylims[1], np.min(y))
        
        ave.append([x,y])
    ave = np.array(ave)
    print ("ave: ", ave.shape)
    
    ave = np.median(ave, axis=0)
    plt.plot(ave[0,:],
             ave[1,:],
             c='red')    

        
    #plt.xlims(xlims)
    #plt.ylims(ylims)
    xmax = max(np.abs(xlims))
    ymax = max(np.abs(ylims))
    plt.plot([-xmax,xmax],
             [0,0],
             '--',
             c='black')
    plt.plot([0,0],
             [-ymax,ymax],
             '--',
             c='black')
    plt.title("Cluster "+str(cluster_id)+ " "+str(cluster_ids[cluster_id].shape[0]))
    
plt.show()

(272450, 25, 2, 2)
(15,)
ave:  (100, 2, 25)
ave:  (100, 2, 25)
ave:  (100, 2, 25)
ave:  (100, 2, 25)
ave:  (100, 2, 25)
ave:  (100, 2, 25)
ave:  (100, 2, 25)
ave:  (100, 2, 25)
ave:  (100, 2, 25)
ave:  (100, 2, 25)
ave:  (100, 2, 25)
ave:  (100, 2, 25)
ave:  (100, 2, 25)
ave:  (100, 2, 25)
ave:  (100, 2, 25)


In [74]:
# colors = plt.cm.viridis(np.linspace(0,1,15))

# frame_id = 0
# for f in range(1):
#     plt.scatter(alldata[frame_id,:,f,0],
#                 alldata[frame_id,:,f,1],
#                c=colors)
# plt.show()

(306873, 15, 2, 2)


In [4]:
#####################################################
#####################################################
#####################################################
def knn_triage(th, embedding):
    from scipy.spatial import cKDTree
    tree = cKDTree(embedding)
    dist, ind = tree.query(embedding, k=51)
    dist = np.sum(dist, 1)
    print (dist)
    per = np.percentile(dist, th)
    idx_keep1 = np.where(dist<= per)[0]
    #print ("idx: ", idx.shape)
    #idx_keep1 = dist <= np.percentile(dist, th)
    
    return idx_keep1

embedding = np.load('/home/cat/umap_x_fit_min_duration_3_dataType_continuous.npy')
embedding = np.load('/home/cat/umap_minDist_0.0_nNeighbor_200.npy')
print ("embedding: ", embedding.shape)

#
th = 80 # % of data to keep
idx_keep = knn_triage(th, embedding)

print ("idx_keep: ", idx_keep.shape)


#####################################################
#####################################################
#####################################################
plt.scatter(embedding[:,0],
            embedding[:,1],
            alpha=.01,
           c='black')

# 
plt.scatter(embedding[idx_keep,0],
            embedding[idx_keep,1],
            alpha=.01,
           c='red')
plt.show()


embedding:  (376728, 2)
[3.79444402 1.80019758 3.51454765 ... 2.56221128 0.97534999 1.73595012]
idx_keep:  (301382,)


In [39]:
from klaus_emd_tools import compute_discretized_and_histograms
min_duration=8
animal_id = 0


(all_continuous,
all_discretized,
all_hist,
all_unique,
angles_thresh,
angles,
angles_discretized,
ang_unique,
ang_hist,
acc_thresh,
acc,
acc_hist,
acc_discretized,
acc_unique) =  compute_discretized_and_histograms(animal_id, 
                                                  min_duration)




... done


In [41]:
print (angles)
print (ang_hist)

[[ 20.117334   0.        20.117334 ...   0.         0.       -23.124643]
 [-44.537777   0.         0.       ...  38.672302   0.         0.      ]
 [  0.         0.         0.       ...  35.941082  49.803715  49.803707]
 ...
 [  0.         0.         0.       ...   0.         0.         0.      ]
 [  0.         0.         0.       ...   0.         0.         0.      ]
 [  0.         0.         0.       ...   0.         0.         0.      ]]
[[0.125 0.5   0.375]
 [0.25  0.625 0.125]
 [0.    0.5   0.5  ]
 ...
 [0.    1.    0.   ]
 [0.    1.    0.   ]
 [0.    1.    0.   ]]
